In [1]:
import gensim
import numpy as np
import pandas as pd
import os

In [2]:
pos = pd.read_csv('Final Data/pos.csv')
pos.columns = ['Eng', 'Hin' , 'Labels']

neg = pd.read_csv('Final Data/neg.csv')
neg.columns = ['Eng', 'Hin' , 'Labels']

data = pd.concat([pos, neg])
data = data.reset_index(drop=True)

In [3]:
X_Hin = np.array(data['Hin'])
Label = np.array(data['Labels'])


In [4]:
from nltk.tokenize import word_tokenize
import nltk

X_token = []
for i in range(len(X_Hin)):
    X_token.append(word_tokenize(X_Hin[i]))
    


In [47]:
import pickle
file = open("VocabDict.obj",'rb')
dict_vocab = pickle.load(file)
file.close()

In [5]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(X_token)]
model_doc2vec = Doc2Vec(documents, vector_size=50, window=2, min_count=3, workers=4)

In [6]:
vocab_List = list(model_doc2vec.wv.vocab.keys())
vocab_df = pd.DataFrame(vocab_List)
vocab_df.to_csv('Vocab.csv', header=None, index=False)

In [7]:
count = 0
new_vocab_List = []
temp_list = []
for i in range(len(vocab_List)):
    count = count + len(vocab_List[i])
    if count > 4000:
        new_vocab_List.append(temp_list)
        temp_list = []
        count = 0
    temp_list.append(vocab_List[i])
new_vocab_List.append(temp_list)

In [39]:
from gensim.models import KeyedVectors
filename = 'GoogleNews-vectors-negative300.bin'
model = KeyedVectors.load_word2vec_format(filename, binary=True)

In [8]:
len(vocab_List)

7440

In [9]:
file = open('Vocab_translated.txt')
Vocab_Eng = file.read().split('\n')

In [10]:
vocab_df['Eng'] = np.array(Vocab_Eng)
vocab_df.columns = ['Hin' , 'Eng']
vocab_df.to_csv('Vocab.csv', header=None, index=False)

In [75]:
model_doc2vec_new = Doc2Vec(vector_size=300, window=2, min_count=3, workers=4, dm=1, alpha=0.025, min_alpha=0.025)
model_doc2vec_new.build_vocab(documents)

for i in range(len(vocab_List)):
    model_doc2vec_new.wv[vocab_List[i]] = dict_vocab[vocab_List[i]]
    
for epoch in range(2):
    model_doc2vec_new.train(documents, total_examples=model_doc2vec_new.corpus_count,
                epochs=model_doc2vec_new.iter)



/Users/karthikeyan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  if __name__ == '__main__':
